In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re   

In [2]:
data = pd.read_csv('merged_datasets/dataset_income_2018-2023_melt.csv')
poverty_guidelines = pd.read_csv('merged_datasets/Federal Poverty Level by Year 2018-2023.csv')

In [6]:
poverty_df = pd.DataFrame(poverty_guidelines)
print(poverty_df.head().to_markdown(index=False, numalign="left", stralign="left"))
print(poverty_df.info())

| Year   | Household Size   | Income    |
|:-------|:-----------------|:----------|
| 2018   | 1                | 12140     |
| 2018   | 2                | 16460     |
| 2018   | 3                | 20780     |
| 2018   | 4                | 25100     |
| 2018   | 5                | 29420     |
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   Year            48 non-null     int64
 1   Household Size  48 non-null     int64
 2   Income          48 non-null     int64
dtypes: int64(3)
memory usage: 1.3 KB
None


In [10]:
# Filter the poverty guidelines to use the threshold for a household size of 4 (a common standard for poverty calculations)
poverty_thresholds = poverty_df[poverty_df['Household Size'] == 4][['Year','Income ']]   
poverty_thresholds = poverty_thresholds.rename(columns={'Income ':'Poverty_Threshold'})
poverty_thresholds.head()

,Year,Poverty_Threshold
3,2018,25100
11,2019,25750
19,2020,26200
27,2021,26500
35,2022,27750


In [33]:
data_df = pd.DataFrame(data)
data_df = data_df.iloc[::2]
data_df.head()

,Year,Category,State,Number_Individuals
0,2018,Total_Households,Alabama,1855184
2,2018,Total_Households_Less_10000,Alabama,203866.3736
4,2018,Total_Households_10000_14999,Alabama,309197.3333
6,2018,Total_Households_15000_24999,Alabama,155897.8151
8,2018,Total_Households_25000_34999,Alabama,183681.5842


In [28]:
#Merge poverty thresholds with the main datas by year
merged_data = pd.merge(data, poverty_thresholds, on='Year', how='left')
merged_data.head()

,Year,Category,State,Number_Individuals,Poverty_Threshold
0,2018,Total_Households,Alabama,1855184,25100
1,2018,Percent_Households_Less_10000,Alabama,9.1,25100
2,2018,Total_Households_Less_10000,Alabama,203866.3736,25100
3,2018,Percent_Households_10000_14999,Alabama,6,25100
4,2018,Total_Households_10000_14999,Alabama,309197.3333,25100


In [31]:
#calculate the poverty threshold measure
#Initialize an empty DataFrame to store the results

poverty_threshold_results = []

#Process each state and year

for year in data['Year'].unique():
    threshold = poverty_thresholds[poverty_thresholds['Year'] == year]['Poverty_Threshold'].values[0]

    #Filter data for the current year
    year_data = merged_data[merged_data['Year'] == year]

    for state in year_data['State'].unique():
        #Filter data for the current state
        state_data = year_data[year_data['State'] == state]

        #Identify income brackets below the poverty line
        poverty_percentage_brackets = state_data[
            (state_data['Category'].str.contains('Percent_Households_')) &
            (state_data['Category'].str.extract(r'(\d+)', expand=False).astype(float) < threshold)
        ]
        
        #Calculate the poverty threshold measure as the sum of relevant percentages
        poverty_threshold_measure = round(poverty_percentage_brackets['Number_Individuals'].astype(float).sum(), 2)

        #Store the results
        poverty_threshold_results.append({
            'Year': year,
            'State': state,
            'Poverty_Threshold_Measure': poverty_threshold_measure
        })
#Convert the results to a DataFrame
poverty_threshold_df = pd.DataFrame(poverty_threshold_results)

#Display the results
poverty_threshold_df   

,Year,State,Poverty_Threshold_Measure
0,2018,Alabama,37.1
1,2018,Alaska,20.8
2,2018,Arizona,28.4
3,2018,Arkansas,37.9
4,2018,California,23.7
...,...,...,...
255,2023,Wisconsin,21.2
256,2023,Wyoming,23.0
257,2023,Puerto Rico,61.6
258,2023,District of Columbia,18.7


In [32]:
output_file_path = '/Users/wmbaluka/OneDrive/Desktop/Rice University Bootcamp/project-3/poverty_threshold_measure.csv'
poverty_threshold_df.to_csv(output_file_path, index=False)
